In [1]:
import PIL
from PIL import Image
import matplotlib

import numpy as np
import pandas as pd

import ast
import re

import json

### **Load Images**

Currently I saved all the images and their information in a csv, which means I don't need to open lots of images when I run my code, but it also means I have to decode the strings in the csv to get my images and labels

In [2]:
train_df = pd.read_csv("C:\\Users\\KurtJi\\OneDrive - University of Illinois - Urbana\\Desktop\\Personal Projects\\data\\inaturalist\\data_csv\\train_dataset.csv")
val_df = pd.read_csv("C:\\Users\\KurtJi\\OneDrive - University of Illinois - Urbana\\Desktop\\Personal Projects\\data\\inaturalist\\data_csv\\val_dataset.csv")
test_df = pd.read_csv("C:\\Users\\KurtJi\\OneDrive - University of Illinois - Urbana\\Desktop\\Personal Projects\\data\\inaturalist\\data_csv\\test_dataset.csv")

In [3]:
val_df.columns

Index(['image_path', 'image_info', 'category_id'], dtype='object')

In [4]:
def get_image_label(cateid: int, categories, type: str):
    # categories should look like this: test_ann['categories']
    # print(categories)
    cate_list = list(categories[0][0].keys())[1:]
    # print(cate_list)
    if type not in cate_list:
        return "please enter correct category level"

    return categories[0][cateid][type]

In [7]:
# Define a function to load and preprocess images
def load_and_preprocess_images(df, type:str, catetype: str):
    image_data = []
    image_info = []
    labels = []

    if type not in ['train', 'val', 'test']:
        print("please enter correct data name!")
        return

    # Validating data
    with open(f"C:\\Users\\KurtJi\\OneDrive - University of Illinois - Urbana\\Desktop\\Personal Projects\\data\\inaturalist\\{type}_mini_sample.json", 'r') as file:
        # Load JSON data from the file
        ann = json.load(file)

    for index, row in df.iterrows():
        image_path = row['image_path']
        info = ast.literal_eval(row['image_info'])
        # get category name
        cate_id = row['category_id']
        categories = ann['categories']
        label = get_image_label(cate_id, categories, catetype)

        # Load and preprocess the image
        image = Image.open(image_path)
        # reshape image
        image = image.resize((224, 224))
        # Apply any necessary preprocessing (resize, normalize, etc.)
        image = np.array(image)  # Convert to NumPy array if needed

        image_data.append(image)
        labels.append(label)

    return np.array(image_data), np.array(image_info), np.array(labels)

In [8]:
train_data, train_info, train_labels = load_and_preprocess_images(train_df, 'train', 'genus')
val_data, val_info, val_labels = load_and_preprocess_images(val_df, 'val', 'genus')
test_data, test_info, test_labels = load_and_preprocess_images(test_df, 'test', 'genus')

In [9]:
np.shape(train_data[99])

(224, 224, 3)

In [10]:
len(val_data)

10000

## **Image Processing**

### **Denoising using fft**

In [11]:
# get number of classes in training set
num_classes = len(np.unique(train_labels))

# get size of image
height, width, channels = 224, 224, 3

In [12]:
num_classes

4836

In [ ]:
%pip install tensorflow

In [13]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Build a simple CNN model
cnn_tf = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channels)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
cnn_tf.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [15]:
# some tf-specific processing
train_data_tf = np.array(train_data)
train_labels_tf = np.array(train_labels)

val_data_tf = np.array(val_data)
val_labels_tf = np.array(val_labels)

test_data_tf = np.array(test_data)
test_labels_tf = np.array(test_labels) 

In [16]:
# Train the model
cnn_tf.fit(train_data_tf, train_labels_tf, epochs=10, validation_data=(val_data_tf, val_labels_tf))

Epoch 1/10


UnimplementedError: Graph execution error:

Detected at node Cast_1 defined at (most recent call last):
  File "c:\Users\KurtJi\miniconda3\lib\runpy.py", line 197, in _run_module_as_main

  File "c:\Users\KurtJi\miniconda3\lib\runpy.py", line 87, in _run_code

  File "c:\Users\KurtJi\miniconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Users\KurtJi\miniconda3\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance

  File "c:\Users\KurtJi\miniconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start

  File "c:\Users\KurtJi\miniconda3\lib\site-packages\tornado\platform\asyncio.py", line 215, in start

  File "c:\Users\KurtJi\miniconda3\lib\asyncio\base_events.py", line 601, in run_forever

  File "c:\Users\KurtJi\miniconda3\lib\asyncio\base_events.py", line 1905, in _run_once

  File "c:\Users\KurtJi\miniconda3\lib\asyncio\events.py", line 80, in _run

  File "c:\Users\KurtJi\miniconda3\lib\site-packages\ipykernel\ke

In [ ]:
# Evaluate the model
accuracy = cnn_tf.evaluate(val_data_tf, val_labels_tf)
print(f'Validation Accuracy: {accuracy[1]}')